En este apartado se hicieron todos las tranformaciones necesarias para las funciones que va a consumir la API. Se tienen como fuentes de datos los archivos ya limpios: user_reviews_cleaned, steam_games_cleaned, user_items_cleaned. Se persigue la eficiencia y ligereza de estas consultas para facilitar el trabajo de la API.

### Importar Bibliotecas

In [3]:
import pandas as pd
import os
import ast

### Cargar el conjunto de datos limpios

In [35]:
df_steam_games = pd.read_csv(r'C:\Users\123la\Documents\GitHub\repositorios de github\PI01_Steam_MLops\data\steam_games_cleaned.csv')
df_user_reviews = pd.read_csv(r'C:\Users\123la\Documents\GitHub\repositorios de github\PI01_Steam_MLops\data\data_reviews_cleaned.csv')
df_user_items = pd.read_csv(r'C:\Users\123la\Documents\GitHub\repositorios de github\PI01_Steam_MLops\data\user_items_cleaned.csv')

### Rutinas

In [4]:
df_reviews = pd.read_csv(r'C:\Users\123la\Documents\GitHub\repositorios de github\PI01_Steam_MLops\data\df_reviews.csv')
df_gastos_items = pd.read_csv(r'C:\Users\123la\Documents\GitHub\repositorios de github\PI01_Steam_MLops\data\df_gastos_items.csv')
df_items_developer = pd.read_csv(r'C:\Users\123la\Documents\GitHub\repositorios de github\PI01_Steam_MLops\data\df_items_developer.csv')
df_playtimeforever = pd.read_csv(r'C:\Users\123la\Documents\GitHub\repositorios de github\PI01_Steam_MLops\data\df_playtimeforever.csv')

### developer 


Esta función recibe como parámetro 'developer', que es la empresa desarrolladora del juego, y devuelve la cantidad de items que desarrolla dicha empresa y el porcentaje de contenido Free por año por sobre el total que desarrolla

In [37]:
def developer(desarrollador):
    # Filtra el dataframe por desarrollador de interés
    data_filtrada = df_items_developer[df_items_developer['developer'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad_por_año = data_filtrada.groupby('release_anio')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('release_anio')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int)

    result_dict = {
        'cantidad_por_año': cantidad_por_año.to_dict(),
        'porcentaje_gratis_por_año': porcentaje_gratis_por_año.to_dict()
    }
    
    return result_dict

In [38]:
desarrollador = 'Mechanical Boss'
developer(desarrollador)

{'cantidad_por_año': {'2016': 2, '2017': 1},
 'porcentaje_gratis_por_año': {'2016': 50, '2017': 0}}

### user data 

In [39]:
df_reviews.head(3)

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,release_anio
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,2009
1,EndAtHallow,http://steamcommunity.com/id/EndAtHallow,1250,No ratings yet,True,2015-01-15,1,2009
2,76561198077432581,http://steamcommunity.com/profiles/76561198077...,1250,No ratings yet,True,2014-12-12,1,2009


In [40]:

def userdata(user_id):
    
    # Filtra por el usuario de interés
    usuario = df_reviews[df_reviews['user_id'] == user_id]
    # Calcula la cantidad de dinero gastado para el usuario de interés
    cantidad_dinero = df_gastos_items[df_gastos_items['user_id']== user_id]['price'].iloc[0]
    # Busca el count_item para el usuario de interés    
    count_items = df_gastos_items[df_gastos_items['user_id']== user_id]['items_count'].iloc[0]
    
    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = usuario['reviews_recommend'].sum()
    # Calcula el total de reviews realizada por todos los usuarios
    total_reviews = len(df_reviews['user_id'].unique())
    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': count_items.astype(int)
    }

In [41]:
user_id = 'EchoXSilence'
userdata(user_id)

{'cantidad_dinero': 189.84,
 'porcentaje_recomendacion': 0.03,
 'total_items': 23}

### UserForGenre 

Se reduce el tamaño del dataset para no exceder el limite de memoria del deploy

In [4]:
df_playtimeforever = df_playtimeforever.head(500)
df_playtimeforever.to_parquet(r'C:\Users\123la\Documents\GitHub\repositorios de github\PI01_Steam_MLops\data\df_playtimeforever.parquet')
df_playtimeforever.to_csv(r'C:\Users\123la\Documents\GitHub\repositorios de github\PI01_Steam_MLops\data\df_playtimeforever.csv')

In [ ]:
def userforgenre(genero):
    # Filtra el dataframe por el género de interés
    data_por_genero = df_playtimeforever[df_playtimeforever['genres'] == genero]
    # Agrupa el dataframe filtrado por usuario y suma la cantidad de horas
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()
    
    # Se hace un diccionario vacío para guardar los datos que se necesitan
    top_users_dict = {}
    for index, row in top_users.iterrows():
        # User info recorre cada fila del top 5 y lo guarda en el diccionario
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']
        }
        top_users_dict[index + 1] = user_info
    return top_users_dict

In [ ]:
userforgenre("Adventure")

{1: {'user_id': 'REBAS_AS_F-T',
  'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 2: {'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami'},
 3: {'user_id': 'Evilutional',
  'user_url': 'http://steamcommunity.com/id/Evilutional'},
 4: {'user_id': 'Sp3ctre', 'user_url': 'http://steamcommunity.com/id/Sp3ctre'},
 5: {'user_id': 'tobscene',
  'user_url': 'http://steamcommunity.com/id/tobscene'}}

##### best_developer_year 

In [ ]:
# Unir los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
df_filtered = df_merged.loc[(df_merged['recommend'] == False) & (df_merged['sentiment_analysis'] == 0),
                           ['year_x', 'developer']]
 
# Renombrar la columna 'year_x' a 'year'
df_filtered = df_filtered.rename(columns={'year_x': 'year'})

# Contar las ocurrencias de cada desarrolladora por año
developer_counts = df_filtered.groupby(['year', 'developer']).size().reset_index(name='count')
    
# Ordenar por 'year' y 'count' en orden descendente
grouped_result = developer_counts.sort_values(by=['year', 'count'], ascending=[False, False])    

# Obtener el top 3 por año
result_df = grouped_result.groupby('year').head(3)    

# Guardar el resultado en un archivo CSV
result_df.to_csv('best_developer_year.csv', index=False)

##### developer_review_analysis

In [ ]:
# Merge de los DataFrames
merged_df = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')

# Agrupar por desarrolladora y sentimiento, y contar las ocurrencias
result_df = merged_df.groupby(['developer', 'sentiment_analysis']).size().reset_index(name='count')

# Pivoteando el DataFrame para tener sentimientos como columnas
result_df = result_df.pivot(index='developer', columns='sentiment_analysis', values='count').reset_index()

# Renombrar las columnas según el formato deseado
result_df.columns = ['developer', 'Negative', 'Neutral', 'Positive']

# Rellenar NaN con 0
result_df = result_df.fillna(0)

# Convertir las columnas a tipo int
result_df[['Negative', 'Neutral', 'Positive']] = result_df[['Negative', 'Neutral', 'Positive']].astype(int)

# Guardar el resultado en un archivo CSV
result_df.to_csv('developer_reviews_analysis.csv', index=False)